<div align="center"> 
<img src='./img/header.png'/>
</div>

## [Global Ice Velocities](https://its-live.jpl.nasa.gov/)
    
The Inter-mission Time Series of Land Ice Velocity and Elevation (ITS_LIVE) project facilitates ice sheet, ice shelf and glacier research by providing a globally comprehensive and temporally dense multi-sensor record of land ice velocity and elevation with low latency.

Scene-pair velocities generated from satellite optical and radar imagery.

* Coverage: All land ice
* Date range: 1985-present
* Resolution: 240m
* Scene-pair separation: 6 to 546 days


---

* If you want to query our API directly using  your own software here is the OpenApi endpoint https://staging.nsidc.org/apps/itslive-search/docs
* For questions about this notebook and the dataset please contact users services at nsidc@nsidc.org


In [ ]:
from map import ItsliveSearchWidget
# vertical means use Jupyter's sidecar panel, horizontal=render in notebook.
m = ItsliveSearchWidget(orientation='vertical')
m.display()

In [ ]:
# you can also define your own parameters and skip the widget.
params = {
    'bbox': [],
    'start': '202',
    'end': '',
    'percent_valid_pixels': '',
    'min interval': '',
    'max interval'
}

In [ ]:
velocitygranules  m.search(params)

In [ ]:
filtered_urls = m.filter_urls(velocitygranules,
                              max_files_per_year=10,
                              months=['November', 'December', 'January', 'February'])
for k in filtered_urls:
    print(k, len(filtered_urls[k]))

In [ ]:
f = m.download_velocity_pairs(urls=filtered_urls)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
filtered_urls['2002']

In [ ]:
files = m.download_velocity_pairs(filtered_urls['2013'])

In [ ]:
import xarray as xr
import glob

velocity_pairs = []
for file in glob.glob('data/*.nc'):
    ds = xr.open_dataset(file)
    velocity_pairs.append(ds)
velocity_pairs[0]

In [ ]:
velocity_pairs[0].img_pair_info.date_center

In [ ]:
if 'Polar_Stereographic' in velocity_pairs[0]:
    print(velocity_pairs[0].Polar_Stereographic.spatial_epsg)

In [ ]:
velocity_pairs[2].plot.scatter(x='x', y='y', hue='v')

In [ ]:
av = xr.align(*velocity_pairs)


In [ ]:
import pandas as pd
da = []

for d in velocity_pairs:
    ds = d.drop_vars(['img_pair_info', 'Polar_Stereographic'])
    ds.coords['time'] = pd.Timestamp(d.img_pair_info.date_center)
    ds.attrs['img_pair_info'] = d.img_pair_info.to_dict()
    ds.attrs['projection'] = d.Polar_Stereographic.to_dict()
    da.append(ds)